#### Setup 
We first set the notebook to display the output from each code block, <br>
then import the required packages and set the file directory.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from shapely.affinity import scale
from shapely.geometry import Point
import pandas as pd
from pygbif.species import name_backbone
import geopandas as gpd

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/data_extended'

#### Get backbone taxonomy
A csv file was exported from the SQL database containing all occurrences for all ungulate species (834.182). A subset was made including only occurrences for which a full species name was available (797.372). As the same species might have been included multiple times under different names, a list was made containing all unique species names. The common taxonomic backbone species key and usage key for these labels were looked up from GBIF and put into a dataframe. <br>

In [ ]:
#Create the dataframes to be concatenated and filtered
occ_all_species = pd.read_csv(file_dir+"/data/SQL_raw_gbif/occurrences_all_species.csv")
df = occ_all_species[occ_all_species['label'].str.contains(" ")]

#Get unique label names
unique_labels=df["label"].unique()

names = []
back_key =[]
remaining_labels=[]

#Get backbone associated species names and taxon keys
for item in unique_labels:
    if "species" in name_backbone(item):
        i = name_backbone(item)['species']
        j = name_backbone(item)['usageKey']
        names.append(i)
        back_key.append(j)
    else:
        remaining_labels.append(item)
        
for item in remaining_labels:
    value=name_backbone(item)['usageKey']
    back_key.append(value)
    names.append(item)
    
#Put into DataFrame
df=pd.DataFrame({"label": unique_labels,"back_key": back_key,"species": names},columns=["label","back_key","species"])

Concatening this dataframe with the dataset with occurrences for all species ensures a common species label for all instances belonging to the same species. Where no match can be found, the dataframe is filled with na values and these rows are dropped. <br>

In [ ]:
#Concatenate with occurrence data, dataframe, drop na's 
df2=pd.merge(occ_all_species,df,how="left",on="label")

df2 = df2[pd.notnull(df2['species'])]
df2 = df2[pd.notnull(df2['decimal_latitude'])]
df2 = df2[pd.notnull(df2['decimal_longitude'])]

print("df2 without na's n.rows:", len(df2.index))

df2["back_key"]=df2["back_key"].astype(int)

Finally, a separate dataframe is stored for each species.

In [ ]:
#list of species
species = df2["species"].unique()
species.sort()

#save separate dataframe for each species as csv file 
for spec in species:
    data=df2.loc[df2['species'] == spec]
    if len(data.index)>= 10:
        spec=spec.replace(" ","_")
        print("%s"%spec, len(data.index))
        data.to_csv(file_dir+'/data/SQL_raw_gbif/%s_raw_data.csv'%spec)

####  Filter occurrences based on criteria.
The species dataframes with raw occurrences from GBIF are then filtered based on having >10 observations, possessing longitude and latitude data with >2 decimals, representing unique longitude-latitude locations, falling within the IUCN species range and having been collected >1900.

In [ ]:
#Open shapefile containing IUCN range of terrestrial mammals
dist = file_dir+"/data/IUCN_mammal_ranges/TERRESTRIAL_MAMMALS.shp"
dist_shp = gpd.read_file(dist)

#create txt file with name of species included after filtering
taxa_list=open(file_dir+'/data/SQL_filtered_gbif/taxa_list.txt',"w")


#Filter occurrences per species
for spec in species:
    
    data=df2.loc[df2['species'] == spec] #select subset of species
    
    # check >10 observations
    if len(data.index)>= 10: 

        spec = spec.replace(" ","_")
        print("processing species %s"%spec)

        data=pd.read_csv(file_dir+'/data/SQL_raw_gbif/%s_raw_data.csv'%spec) #load in data
        
        ###################################################
        # check number of decimals longitude and latitude #
        ###################################################
        str_lat=(pd.Series.tolist(data["decimal_latitude"].astype(str)))
        str_lon=(pd.Series.tolist(data["decimal_longitude"].astype(str)))
        dec_lat=[]
        dec_lon=[]

        for i in range(len(str_lat)):
    
            if "e" in str_lat[i]:
                str_lat[i]="0.00"
                decla = str_lat[i].split(".")[1]
                print(i, str_lat[i],decla)
                dec_lat.append(int(len(decla)))
            else:
                decla = str_lat[i].split(".")[1]
                dec_lat.append(int(len(decla)))
                
        for i in range(len(str_lon)):
            declo=str_lon[i].split(".")[1]
            dec_lon.append(int(len(declo)))
    
        data["dec_lat"]=dec_lat
        data["dec_lon"]=dec_lon

        # filter only include those with min. 2 points
        data=data[data["dec_lat"] >= 2]
        data=data[data["dec_lon"] >= 2]
        print("length only including lon-lat 2 decimals",len(data.index))

        data['coordinates'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
        data['lonlat'] = list(zip(data["decimal_longitude"], data["decimal_latitude"]))
        data['coordinates'] = data["coordinates"].apply(Point)

        
        #########################################
        # only keep records with unique lon-lat #
        #########################################
        
        data = data.drop_duplicates('lonlat')
        print("length unique lon-lat",len(data.index))

       
        ###############################################
        # only keep records falling in IUNC range #
        ###############################################
        
        speci=spec.replace("_"," ")
        dist_shp_spec = dist_shp[dist_shp["binomial"]== "%s"%speci]
        poly_spec = dist_shp_spec[["geometry"]]
        
        # merge the polygons
        iucn_poly_spec= poly_spec.unary_union
        Q3 = iucn_poly_spec.simplify(0.3)
        Q3 #inspect polygon

        if Q3.is_valid== False:
            Q3 = Q3.buffer(0)

        condition_list=[]

        for point in data["coordinates"]:
            output= point.within(Q3)
            condition_list.append(output)

        #keep records that are in species range
        data["in_dist_polygon"]=condition_list
        data2=data[data.in_dist_polygon == True]
        print("length in species dist polygon",len(data2.index))

        #############################
        # Only keep records > 1900  #
        #############################
        
        data2['event_date'] = pd.to_datetime(data2['event_date']) # set date column to datetime format to extract year
        data2['year'] = data2['event_date'].dt.year
        data2['month']= data2['event_date'].dt.month

        #set date column to datetime format and extract year
        data2['event_date'] = pd.to_datetime(data2['event_date'])
        data2['year'] = data2['event_date'].dt.year
        data2['month']= data2['event_date'].dt.month

        #only include observations >1900
        data3=data2[data2.year >= 1900]
        print("length observationas >1900", len(data3.index))
        
        
        # check >10 observations
        if len(data3.index)>=10:
            #save to csv
            data3.to_csv(file_dir+'/data/SQL_filtered_gbif/%s_filtered_data.csv'%spec)
            taxa_list.write(spec+"\n")
            
#close text file
taxa_list.close()
# next species!